In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
import pandas as pd
train=pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test=pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
submission=pd.read_csv("/kaggle/input/spaceship-titanic/sample_submission.csv")

In [3]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [4]:
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [5]:
submission

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False
...,...,...
4272,9266_02,False
4273,9269_01,False
4274,9271_01,False
4275,9273_01,False


# Preprocess data

In [6]:
train.corr()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
Age,1.000000,0.068723,0.130421,0.033133,0.123970,0.101007,-0.075026
RoomService,0.068723,1.000000,-0.015889,0.054480,0.010080,-0.019581,-0.244611
FoodCourt,0.130421,-0.015889,1.000000,-0.014228,0.221891,0.227995,0.046566
ShoppingMall,0.033133,0.054480,-0.014228,1.000000,0.013879,-0.007322,0.010141
Spa,0.123970,0.010080,0.221891,0.013879,1.000000,0.153821,-0.221131
VRDeck,0.101007,-0.019581,0.227995,-0.007322,0.153821,1.000000,-0.207075
Transported,-0.075026,-0.244611,0.046566,0.010141,-0.221131,-0.207075,1.000000


In [7]:
train.columns 

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

Passenger id is an id so we should keep it <br>
Home Planet can be a deciding factor so we can convert it to numerical <br>
Cryosleep too determines whether transportation was done or not- convert to numerical <br>
Cabin lets see <br>
Destination - lets see <br>
Age - lets see <br>
VIP -lets see <br>
All expenses can be combined into a single feature - expense (RoomService	FoodCourt	ShoppingMall	Spa	VRDeck) <br>
Name is irrelevant - remove it <br>
Transported - label to be predicted


# Home Planet

In [8]:
train["HomePlanet"].unique()

array(['Europa', 'Earth', 'Mars', nan], dtype=object)

In [9]:
onehot_HomePlanet=pd.get_dummies(train, columns = ["HomePlanet"])

In [10]:
onehot_HomePlanet

,PassengerId,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars
0,0001_01,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0,1,0
1,0002_01,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,1,0,0
2,0003_01,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0,1,0
3,0003_02,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0,1,0
4,0004_01,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False,0,1,0
8689,9278_01,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,1,0,0
8690,9279_01,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,1,0,0
8691,9280_01,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False,0,1,0


In [11]:
train

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


# Cryosleep

In [12]:
train["CryoSleep"].unique()

array([False, True, nan], dtype=object)

Lets label True as 1 and False as 0; ignore nan and assume False ie 0

In [13]:
def fill_cryo(row):
    if row["CryoSleep"]==True:
        return 1
    else:
        return 0

In [14]:
train_new=onehot_HomePlanet


In [15]:
train_new["Cryo"]=train_new.apply(lambda row: fill_cryo(row), axis=1)
train_new

,PassengerId,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Cryo
0,0001_01,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0,1,0,0
1,0002_01,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,1,0,0,0
2,0003_01,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0,1,0,0
3,0003_02,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0,1,0,0
4,0004_01,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False,0,1,0,0
8689,9278_01,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,1,0,0,1
8690,9279_01,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,1,0,0,0
8691,9280_01,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False,0,1,0,0


In [16]:
train_new =train_new.drop("CryoSleep", axis =1)

In [17]:
train_new

,PassengerId,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Cryo
0,0001_01,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0,1,0,0
1,0002_01,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,1,0,0,0
2,0003_01,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0,1,0,0
3,0003_02,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0,1,0,0
4,0004_01,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False,0,1,0,0
8689,9278_01,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,1,0,0,1
8690,9279_01,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,1,0,0,0
8691,9280_01,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False,0,1,0,0


# Cabin

In [18]:
print(train["Cabin"].unique())
print(train["Cabin"].value_counts())

['B/0/P' 'F/0/S' 'A/0/S' ... 'G/1499/S' 'G/1500/S' 'E/608/S']
G/734/S     8
G/109/P     7
B/201/P     7
G/1368/P    7
G/981/S     7
           ..
G/556/P     1
E/231/S     1
G/545/S     1
G/543/S     1
F/947/P     1
Name: Cabin, Length: 6560, dtype: int64


Break the cabin into 3 parts: deck / num/ side

In [19]:
train_cabin=pd.DataFrame().assign(Cabin=train['Cabin'], Transported=train['Transported'])
train_cabin


,Cabin,Transported
0,B/0/P,False
1,F/0/S,True
2,A/0/S,False
3,A/0/S,False
4,F/1/S,True
...,...,...
8688,A/98/P,False
8689,G/1499/S,False
8690,G/1500/S,True
8691,E/608/S,False


In [20]:
train_cabin[['deck', 'num', 'side']] = train_cabin['Cabin'].str.split("/",expand=True)
train_cabin

,Cabin,Transported,deck,num,side
0,B/0/P,False,B,0,P
1,F/0/S,True,F,0,S
2,A/0/S,False,A,0,S
3,A/0/S,False,A,0,S
4,F/1/S,True,F,1,S
...,...,...,...,...,...
8688,A/98/P,False,A,98,P
8689,G/1499/S,False,G,1499,S
8690,G/1500/S,True,G,1500,S
8691,E/608/S,False,E,608,S


In [21]:
train_cabin.describe()

,Cabin,Transported,deck,num,side
count,8494,8693,8494,8494,8494
unique,6560,2,8,1817,2
top,G/734/S,True,F,82,S
freq,8,4378,2794,28,4288


In [22]:
#replace nan with top values from df.describe()
import numpy as np
train_cabin['deck'] = train_cabin['deck'].replace(np.nan, "F")
train_cabin['num'] = train_cabin['num'].replace(np.nan, "82")
train_cabin['side'] = train_cabin['side'].replace(np.nan, "S")
train_cabin

,Cabin,Transported,deck,num,side
0,B/0/P,False,B,0,P
1,F/0/S,True,F,0,S
2,A/0/S,False,A,0,S
3,A/0/S,False,A,0,S
4,F/1/S,True,F,1,S
...,...,...,...,...,...
8688,A/98/P,False,A,98,P
8689,G/1499/S,False,G,1499,S
8690,G/1500/S,True,G,1500,S
8691,E/608/S,False,E,608,S


In [23]:
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# conversion functions to numerical categories
def conv_num(row):
    return int(row["num"])
def conv_side(row):
    if row["side"]=="S":
        return 0
    else:
        return 1
train_cabin['deck']= label_encoder.fit_transform(train_cabin['deck'])
train_cabin["num"]=train_cabin.apply(lambda row: conv_num(row), axis=1)
train_cabin["side"]=train_cabin.apply(lambda row: conv_side(row), axis=1)
train_cabin



,Cabin,Transported,deck,num,side
0,B/0/P,False,1,0,1
1,F/0/S,True,5,0,0
2,A/0/S,False,0,0,0
3,A/0/S,False,0,0,0
4,F/1/S,True,5,1,0
...,...,...,...,...,...
8688,A/98/P,False,0,98,1
8689,G/1499/S,False,6,1499,0
8690,G/1500/S,True,6,1500,0
8691,E/608/S,False,4,608,0


In [24]:
train_cabin=train_cabin.drop("Cabin", axis=1)
train_cabin

,Transported,deck,num,side
0,False,1,0,1
1,True,5,0,0
2,False,0,0,0
3,False,0,0,0
4,True,5,1,0
...,...,...,...,...
8688,False,0,98,1
8689,False,6,1499,0
8690,True,6,1500,0
8691,False,4,608,0


In [25]:
train_cabin.corr()

,Transported,deck,num,side
Transported,1.000000,-0.114941,-0.044012,-0.101397
deck,-0.114941,1.000000,0.515466,0.025316
num,-0.044012,0.515466,1.000000,0.059589
side,-0.101397,0.025316,0.059589,1.000000


When testing we can try dropping each of these columns to try if it matters at all

In [26]:
train_new

,PassengerId,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Cryo
0,0001_01,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0,1,0,0
1,0002_01,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,1,0,0,0
2,0003_01,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0,1,0,0
3,0003_02,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0,1,0,0
4,0004_01,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False,0,1,0,0
8689,9278_01,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,1,0,0,1
8690,9279_01,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,1,0,0,0
8691,9280_01,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False,0,1,0,0


In [27]:
train_new=train_new.drop("Cabin", axis=1)
train_new["deck"]=train_cabin["deck"]
train_new["num"]=train_cabin["num"]
train_new["side"]=train_cabin["side"]
train_new

,PassengerId,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Cryo,deck,num,side
0,0001_01,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0,1,0,0,1,0,1
1,0002_01,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,1,0,0,0,5,0,0
2,0003_01,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0,1,0,0,0,0,0
3,0003_02,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0,1,0,0,0,0,0
4,0004_01,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,1,0,0,0,5,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False,0,1,0,0,0,98,1
8689,9278_01,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,1,0,0,1,6,1499,0
8690,9279_01,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,1,0,0,0,6,1500,0
8691,9280_01,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False,0,1,0,0,4,608,0


# Destination

In [28]:
train_new["Destination"].unique()

array(['TRAPPIST-1e', 'PSO J318.5-22', '55 Cancri e', nan], dtype=object)

In [29]:
train_new["Destination"].describe()

count            8511
unique              3
top       TRAPPIST-1e
freq             5915
Name: Destination, dtype: object

In [30]:
# Replace nan with top value
train_new["Destination"] = train_new["Destination"].replace(np.nan, "TRAPPIST-1e")
#Convert into numerical category
train_new=pd.get_dummies(train_new, columns = ["Destination"])


In [31]:
train_new

,PassengerId,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Cryo,deck,num,side,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0001_01,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0,1,0,0,1,0,1,0,0,1
1,0002_01,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,1,0,0,0,5,0,0,0,0,1
2,0003_01,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0,1,0,0,0,0,0,0,0,1
3,0003_02,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0,1,0,0,0,0,0,0,0,1
4,0004_01,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,1,0,0,0,5,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False,0,1,0,0,0,98,1,1,0,0
8689,9278_01,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,1,0,0,1,6,1499,0,0,1,0
8690,9279_01,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,1,0,0,0,6,1500,0,0,0,1
8691,9280_01,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False,0,1,0,0,4,608,0,1,0,0


# Age

In [32]:
train.corr()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
Age,1.000000,0.068723,0.130421,0.033133,0.123970,0.101007,-0.075026
RoomService,0.068723,1.000000,-0.015889,0.054480,0.010080,-0.019581,-0.244611
FoodCourt,0.130421,-0.015889,1.000000,-0.014228,0.221891,0.227995,0.046566
ShoppingMall,0.033133,0.054480,-0.014228,1.000000,0.013879,-0.007322,0.010141
Spa,0.123970,0.010080,0.221891,0.013879,1.000000,0.153821,-0.221131
VRDeck,0.101007,-0.019581,0.227995,-0.007322,0.153821,1.000000,-0.207075
Transported,-0.075026,-0.244611,0.046566,0.010141,-0.221131,-0.207075,1.000000


Age is not correlated so let it be

# VIP

In [33]:
def fill_VIP(row):
    if row["VIP"]==True:
        return 1
    else:
        return 0

In [34]:
train_new["VIP"]=train_new.apply(lambda row: fill_VIP(row), axis=1)
train_new

,PassengerId,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Cryo,deck,num,side,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0001_01,39.0,0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0,1,0,0,1,0,1,0,0,1
1,0002_01,24.0,0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,1,0,0,0,5,0,0,0,0,1
2,0003_01,58.0,1,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0,1,0,0,0,0,0,0,0,1
3,0003_02,33.0,0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0,1,0,0,0,0,0,0,0,1
4,0004_01,16.0,0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,1,0,0,0,5,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,41.0,1,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False,0,1,0,0,0,98,1,1,0,0
8689,9278_01,18.0,0,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,1,0,0,1,6,1499,0,0,1,0
8690,9279_01,26.0,0,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,1,0,0,0,6,1500,0,0,0,1
8691,9280_01,32.0,0,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False,0,1,0,0,4,608,0,1,0,0


# Expenses

In [35]:
def fill_expense(row):
    return row["RoomService"]+row["FoodCourt"]+ row["ShoppingMall"]+ row["Spa"]+row["VRDeck"]

In [36]:
train_new["Exp"]=train_new.apply(lambda row: fill_expense(row), axis=1)
train_new =train_new.drop("RoomService", axis =1)
train_new =train_new.drop("FoodCourt", axis =1)
train_new =train_new.drop("ShoppingMall", axis =1)
train_new =train_new.drop("Spa", axis =1)
train_new =train_new.drop("VRDeck", axis =1)

Y= train_new["Transported"]
X=train_new.drop("Transported", axis =1)
X=X.drop("PassengerId", axis =1)
X=X.drop("Name", axis =1)

In [37]:
X

,Age,VIP,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Cryo,deck,num,side,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Exp
0,39.0,0,0,1,0,0,1,0,1,0,0,1,0.0
1,24.0,0,1,0,0,0,5,0,0,0,0,1,736.0
2,58.0,1,0,1,0,0,0,0,0,0,0,1,10383.0
3,33.0,0,0,1,0,0,0,0,0,0,0,1,5176.0
4,16.0,0,1,0,0,0,5,1,0,0,0,1,1091.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,41.0,1,0,1,0,0,0,98,1,1,0,0,8536.0
8689,18.0,0,1,0,0,1,6,1499,0,0,1,0,0.0
8690,26.0,0,1,0,0,0,6,1500,0,0,0,1,1873.0
8691,32.0,0,0,1,0,0,4,608,0,1,0,0,4637.0


In [38]:
Y

0       False
1        True
2       False
3       False
4        True
        ...  
8688    False
8689    False
8690     True
8691    False
8692     True
Name: Transported, Length: 8693, dtype: bool

# Training

In [39]:
import numpy as np
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X_1=imp.fit_transform(X)

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_1,Y, test_size=0.25, random_state = 0)

In [41]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

model=SVC(C=1000, kernel="rbf")
model.fit(X_train, Y_train)
Y_pred=model.predict(X_test)
acc=accuracy_score(Y_test, Y_pred)
print(acc)

0.7065317387304508


In [42]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

model=LinearSVC(C=1000)
model.fit(X_train, Y_train)
Y_pred=model.predict(X_test)
acc=accuracy_score(Y_test, Y_pred)
print(acc)

0.4954001839926403


/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [43]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

model=GaussianNB()
model.fit(X_train, Y_train)
Y_pred=model.predict(X_test)
acc=accuracy_score(Y_test, Y_pred)
print(acc)

0.7134314627414904


In [44]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=1000)
model.fit(X_train, Y_train)
Y_pred=model.predict(X_test)
acc=accuracy_score(Y_test, Y_pred)
print(acc)

0.7069917203311867


# Drop age and expenses

In [45]:
X_2=X.drop("Exp", axis=1)
import numpy as np
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X_2=imp.fit_transform(X_2)

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_2,Y, test_size=0.25, random_state = 0)

In [47]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

model2=GaussianNB()
model2.fit(X_train, Y_train)
Y_pred=model2.predict(X_test)
acc=accuracy_score(Y_test, Y_pred)
print(acc)

0.6936522539098436


In [48]:
from sklearn.neighbors import KNeighborsClassifier
model2 = KNeighborsClassifier(n_neighbors=3)
model2.fit(X_train, Y_train)
Y_pred=model2.predict(X_test)
acc=accuracy_score(Y_test, Y_pred)
print(acc)

0.5565777368905244


In [49]:
X.columns

Index(['Age', 'VIP', 'HomePlanet_Earth', 'HomePlanet_Europa',
       'HomePlanet_Mars', 'Cryo', 'deck', 'num', 'side',
       'Destination_55 Cancri e', 'Destination_PSO J318.5-22',
       'Destination_TRAPPIST-1e', 'Exp'],
      dtype='object')

In [50]:
X_3=X.drop("VIP", axis=1)
X_3=X_3.drop("num", axis=1)
X_3=X_3.drop("Age", axis=1)
#X_3=X_3.drop("Exp", axis=1)
#X_3=X_3.drop("Destination_55 Cancri e", axis=1)
#Exp and Destination_55 Cancri e removal downgrade the accuracy
X_3=X_3.drop("HomePlanet_Earth", axis=1)
X_3=X_3.drop("HomePlanet_Europa", axis=1)
#X_3=X_3.drop("HomePlanet_Mars", axis=1)
#X_3=X_3.drop("Cryo", axis=1)
#X_3=X_3.drop("deck", axis=1)
X_3=X_3.drop("Destination_TRAPPIST-1e", axis=1)
#X_3=X_3.drop("Destination_PSO J318.5-22", axis=1)
#X_3=X_3.drop("side", axis=1)
# Destination_TRAPPIST-1e removal gave a significant performance improvement
import numpy as np
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X_3=imp.fit_transform(X_3)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_3,Y, test_size=0.25, random_state = 0)

In [51]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

model3=GaussianNB()
model3.fit(X_train, Y_train)
Y_pred=model3.predict(X_test)
acc=accuracy_score(Y_test, Y_pred)
print(acc)

0.7125114995400184


In [52]:
from sklearn.neighbors import KNeighborsClassifier
model2 = KNeighborsClassifier(n_neighbors=10)
model2.fit(X_train, Y_train)
Y_pred=model2.predict(X_test)
acc=accuracy_score(Y_test, Y_pred)
print(acc)

0.7290708371665133


In [53]:
import xgboost as xgb
model=xgb.XGBClassifier()
model.fit(X_train, Y_train)
Y_pred=model.predict(X_test)
acc=accuracy_score(Y_test, Y_pred)
print(acc)

0.7419503219871205


In [54]:
from sklearn.ensemble import GradientBoostingClassifier
model=GradientBoostingClassifier()
model.fit(X_train, Y_train)
Y_pred=model.predict(X_test)
acc=accuracy_score(Y_test.values, Y_pred)
print(acc)

0.7465501379944802


In [55]:
submission

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False
...,...,...
4272,9266_02,False
4273,9269_01,False
4274,9271_01,False
4275,9273_01,False


In [56]:
test

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale


In [57]:
test_new=pd.get_dummies(test, columns = ["HomePlanet"])

In [58]:
test_new["Cryo"]=test_new.apply(lambda row: fill_cryo(row), axis=1)

In [59]:
test_new =test_new.drop("CryoSleep", axis =1)

In [60]:
test_cabin=pd.DataFrame().assign(Cabin=test['Cabin'])
test_cabin[['deck', 'num', 'side']] = test_cabin['Cabin'].str.split("/",expand=True)
#replace nan with top values from df.describe()
import numpy as np
test_cabin['deck'] = test_cabin['deck'].replace(np.nan, "F")
test_cabin['num'] = test_cabin['num'].replace(np.nan, "82")
test_cabin['side'] = test_cabin['side'].replace(np.nan, "S")

from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# conversion functions to numerical categories
def conv_num(row):
    return int(row["num"])
def conv_side(row):
    if row["side"]=="S":
        return 0
    else:
        return 1
test_cabin['deck']= label_encoder.fit_transform(test_cabin['deck'])
test_cabin["num"]=test_cabin.apply(lambda row: conv_num(row), axis=1)
test_cabin["side"]=test_cabin.apply(lambda row: conv_side(row), axis=1)

test_new=test_new.drop("Cabin", axis=1)
test_new["deck"]=test_cabin["deck"]
test_new["num"]=test_cabin["num"]
test_new["side"]=test_cabin["side"]

# Replace nan with top value
test_new["Destination"] = test_new["Destination"].replace(np.nan, "TRAPPIST-1e")
#Convert into numerical category
test_new=pd.get_dummies(test_new, columns = ["Destination"])

test_new["VIP"]=test_new.apply(lambda row: fill_VIP(row), axis=1)
test_new["Exp"]=test_new.apply(lambda row: fill_expense(row), axis=1)
test_new =test_new.drop("RoomService", axis =1)
test_new =test_new.drop("FoodCourt", axis =1)
test_new =test_new.drop("ShoppingMall", axis =1)
test_new =test_new.drop("Spa", axis =1)
test_new =test_new.drop("VRDeck", axis =1)
print(test_new)
X=test_new
#X=X.drop("PassengerId", axis =1)
X=X.drop("Name", axis =1)

X_3=X.drop("VIP", axis=1)
X_3=X_3.drop("num", axis=1)
X_3=X_3.drop("Age", axis=1)
#X_3=X_3.drop("Exp", axis=1)
#X_3=X_3.drop("Destination_55 Cancri e", axis=1)
#Exp and Destination_55 Cancri e removal downgrade the accuracy
X_3=X_3.drop("HomePlanet_Earth", axis=1)
X_3=X_3.drop("HomePlanet_Europa", axis=1)
#X_3=X_3.drop("HomePlanet_Mars", axis=1)
#X_3=X_3.drop("Cryo", axis=1)
#X_3=X_3.drop("deck", axis=1)
X_3=X_3.drop("Destination_TRAPPIST-1e", axis=1)
#X_3=X_3.drop("Destination_PSO J318.5-22", axis=1)
#X_3=X_3.drop("side", axis=1)
# Destination_TRAPPIST-1e removal gave a significant performance improvement

import numpy as np
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')




     PassengerId   Age  VIP              Name  HomePlanet_Earth  \
0        0013_01  27.0    0   Nelly Carsoning                 1   
1        0018_01  19.0    0    Lerome Peckers                 1   
2        0019_01  31.0    0   Sabih Unhearfus                 0   
3        0021_01  38.0    0  Meratz Caltilter                 0   
4        0023_01  20.0    0   Brence Harperez                 1   
...          ...   ...  ...               ...               ...   
4272     9266_02  34.0    0       Jeron Peter                 1   
4273     9269_01  42.0    0     Matty Scheron                 1   
4274     9271_01   NaN    0       Jayrin Pore                 0   
4275     9273_01   NaN    0    Kitakan Conale                 0   
4276     9277_01  43.0    0  Lilace Leonzaley                 1   

      HomePlanet_Europa  HomePlanet_Mars  Cryo  deck   num  side  \
0                     0                0     1     6     3     0   
1                     0                0     0     5     4 

In [61]:
X_3=X_3.drop("PassengerId", axis=1)
X_3=imp.fit_transform(X_3)


In [62]:
y_pred = model.predict(X_3)

sub=pd.DataFrame({'Transported':y_pred.astype(bool)},index=test.index)

sub.head()

,Transported
0,True
1,False
2,True
3,False
4,False


In [63]:
submission

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False
...,...,...
4272,9266_02,False
4273,9269_01,False
4274,9271_01,False
4275,9273_01,False


In [64]:
submission["Transported"]=sub["Transported"]
submission

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,False
4,0023_01,False
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,False


In [65]:
submission.to_csv('submission.csv', index=False)